<a href="https://colab.research.google.com/github/ddoyungyung/Final-Team5/blob/main/%EB%A9%8B%EC%9F%81%EC%9D%B4_%EC%82%AC%EC%9E%90%EC%B2%98%EB%9F%BC_final_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EC%86%8C%EC%9C%A43(pos)ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# noun_adj_pairs_raw(pos) 파일 먼저 이용!

In [ ]:
#정원님 미로 positive 리뷰  포스트잇 내용 사용 + bert모델이용한 라벨링 재정의할 때(neg) 만든 시설/어메니티 라벨링 그대로 사용

#attraction: museum, park,view,beautiful,amazing,photo
#eat:restaurant, bar
#transport: train, subway,metro, bus,station,metre,underground, transport, access, location, centre, position,tube
#service: 'lovely', 'friendly', 'staff', 'helpful', 'breakfast', 'tidy', 'polite', 'reception', 'value', 'quality', 'check', 'experience', 'response',    'food', 'drink', 'egg', 'selection', 'desk', 'day','coffee'  #부정이랑 맞추기, poor / slow / old/ more 은 제외!
#facility_amenity:'water', 'hot', 'cold', 'condition', 'balcony', 'wifi','desk','hotel', 'room', 'bed', 'water', 'furniture', 'building', 'carpet', 'window', 'mattress',    'wifi', 'connection', 'elevator', 'lift', 'fridge', 'lighting', 'tub', 'shower', 'air','level','audio','recording'

In [ ]:
import pandas as pd
from collections import Counter, defaultdict
import re
import string
import spacy
import os
from tqdm import tqdm


df = pd.read_csv('/content/noun_adj_pairs_raw(pos).csv')

# 정원님이 만든 키워드 딕셔너리화
keywords_dict ={
    'attraction' : ['museum', 'park', 'view', 'beautiful', 'amazing'],
    'eat' : ['restaurant', 'bar'],
    'transport' : ['great', 'train', 'subway', 'metro', 'bus', 'station', 'metre', 'underground', 'transport', 'access', 'location', 'centre', 'position', 'tube'],
    'service' : ['lovely', 'friendly', 'staff', 'helpful', 'breakfast', 'tidy', 'polite', 'reception','value','quality','check','experience','response','food','drink','egg','selection','desk','day','coffee'],
    'facility_amenity' : ['water','hot','cold','condition','balcony', 'wifi','desk','hotel','room','bed','water','furniture','building','carpet','window','mattress','wifi','connection','elevator','lift','fridge','lighting','tub','shower','air','level','audio','recording'],
    'roomsize' : ['tidy']
}

# 라벨링 함수를 만들어보자!
def assign_label(text):
    if pd.isna(text): # 혹시 텍스트가 비어있는 경우 (NaN)를 대비
        return 'Not Applicable'
    text = str(text).lower() # 대소문자 상관없이 찾으려면 소문자로 다 바꿔주기. 및 문자열!

    for label, keywords in keywords_dict.items():
        for keyword in keywords:
            if keyword in text:
                    return label # 키워드를 찾으면 해당 라벨을 반환하고 끝!
    return 'Other/General' # 키워드를 하나도 못 찾으면 'Other/General'키워드


# 형용사와 명사를 띄어쓰기로 연결해서 하나의 문장처럼 만들기
df['combined_text'] = df['adj'].astype(str) + ' ' + df['noun'].astype(str)

# 새로운 'label' 컬럼을 추가해서 함수를 적용
df['label'] = df['combined_text'].apply(assign_label)

print(df[['adj', 'noun', 'combined_text', 'label']])

             adj       noun     combined_text          label
0           good   location     good location      transport
1         lovely       food       lovely food        service
2       friendly       food     friendly food        service
3           high    quality      high quality        service
4            big       park          big park     attraction
...          ...        ...               ...            ...
643756     super     market      super market  Other/General
643757     fress      fruit       fress fruit  Other/General
643758      good      acess        good acess  Other/General
643759    public  transport  public transport      transport
643760   helpful      staff     helpful staff        service

[643761 rows x 4 columns]


bert 파일 이용

In [ ]:
# 리스트 만들기

attraction_keywords = ['museum', 'park', 'view', 'beautiful', 'amazing']
eat_keywords = ['restaurant', 'bar']
transport_keywords = ['great', 'train', 'subway', 'metro', 'bus', 'station', 'metre', 'underground', 'transport', 'access', 'location', 'centre', 'position', 'tube']
service_keywords = ['lovely', 'friendly', 'staff', 'helpful', 'breakfast', 'tidy', 'polite', 'reception','value','quality','check','experience','response','food','drink','egg','selection','desk','day','coffee']
facility_amenity_kewords = ['water','hot','cold','condition','balcony', 'wifi','desk','hotel','room','bed','water','furniture','building','carpet','window','mattress','wifi','connection','elevator','lift','fridge','lighting','tub','shower','air','level','audio','recording']
roomsize_keywords = ['tidy']

# 모든 수동 키워드를 하나의 세트로 통합
manual_keywords = set(attraction_keywords + eat_keywords + transport_keywords + service_keywords + roomsize_keywords)

# 2. BERT 긍정 리뷰 분석 파일 로드
df_bert_pos  = pd.read_csv("/content/positive_reviews_bert_analysis_1.csv")

# 3. BERT 파일에서 모든 단어 추출
all_words = []
for pair_string in df_bert_pos['word_pair']:
    try:
        pair = eval(pair_string)
        if isinstance(pair, tuple) and len(pair) == 2:
            all_words.extend(pair)
    except (SyntaxError, NameError):
        continue

# 4. 수동 키워드 리스트에 없는 단어(새로운 키워드 후보) 찾기
new_keywords_candidates = [word for word in all_words if word not in manual_keywords]

# 5. 새로운 키워드 후보의 빈도수 계산 및 상위 20개 출력
new_keyword_counts = Counter(new_keywords_candidates)

print("\n--- BERT 분석에서 발견된 새로운 키워드 후보 (상위 20개) ---")
for word, count in new_keyword_counts.most_common(20):
    print(f"  - {word}: {count:,}")



--- BERT 분석에서 발견된 새로운 키워드 후보 (상위 20개) ---
  - very: 32,574
  - good: 24,968
  - room: 23,749
  - nice: 15,093
  - comfortable: 13,011
  - clean: 12,177
  - hotel: 11,667
  - excellent: 10,475
  - bed: 8,970
  - it: 5,320
  - perfect: 4,372
  - quiet: 3,831
  - service: 3,630
  - really: 3,398
  - modern: 2,850
  - spacious: 2,842
  - fantastic: 2,832
  - comfy: 2,731
  - so: 2,686
  - area: 2,485


word_pair컬럼 사용해서, 빈도수 찾아내기!

In [ ]:
# 분석 대상 형용사 정의
# 명사에 대한 꾸밈에 집중할 만한 형용사들
# 'room', 'htel', 'it','service', 'bed','area' 등은 명사에 더 가까워서 이 리스트에서 제외
target_adjectives = ['very','good', 'bad', 'nice','comfortable','clean','excellent','perfect','quiet','really','modern','spacious','fantastic','comfy','so']

# 'word_pair' 컬럼에서 형용사와 명사 분리
def parse_word_pair(pair_string):
    try:
        if pair_string.startswith('(') and pair_string.endswith(')'): #'string' 리터럴이 아닌 실제 파이썬 튜플 형태인지 확인
            pair = eval(pair_string)
            if isinstance(pair, tuple) and len(pair) == 2:
                adj = str(pair[0]).strip() # 튜플 요소의 타입도 확인하여 불필요한 공백 제거
                noun = str(pair[1]).strip()
                return (adj, noun)
        return None
    except (SyntaxError, NameError, TypeError): # TypeError도 추가해서 좀 더 견고하게!
        return None

df_bert_pos['word_pair_tuple'] = df_bert_pos['word_pair'].apply(parse_word_pair)
df_bert_pos.dropna(subset=['word_pair_tuple'], inplace=True) # 유효하지 않은 쌍 제거

# 해당 타겟 형용사가 포함된 쌍만 필터링
for target_adj in target_adjectives:
    filtered_pairs = df_bert_pos[df_bert_pos['word_pair_tuple'].apply(lambda x: x[0] == target_adj)] # word_pair_tuple은 (adj, noun) 형태의 튜플이므로, adj가 target_adj와 같은지 확인

    # 해당 형용사가 꾸며주는 명사들만 추출
    nouns_for_adj = [pair[1] for pair in filtered_pairs['word_pair_tuple']]

    if nouns_for_adj: # 명사가 하나라도 있으면 출력
        noun_counts_for_adj = Counter(nouns_for_adj).most_common(10)
        print(f"'{target_adj}'이(가) 꾸며주는 명사 (총 {len(nouns_for_adj):,}개) :")
        for noun, count in noun_counts_for_adj:
            print(f"  - {noun}: {count:,}")
        print("--------------------------------------------------")

'very'이(가) 꾸며주는 명사 (총 32,574개) :
  - friendly: 4,366
  - helpful: 4,036
  - good: 3,924
  - clean: 3,220
  - comfortable: 3,159
  - nice: 2,704
  - comfy: 772
  - quiet: 678
  - convenient: 626
  - pleasant: 514
--------------------------------------------------
'good'이(가) 꾸며주는 명사 (총 15,487개) :
  - location: 2,980
  - breakfast: 1,306
  - staff: 978
  - room: 824
  - value: 797
  - hotel: 630
  - service: 406
  - bed: 323
  - size: 312
  - facility: 297
--------------------------------------------------
'bad'이(가) 꾸며주는 명사 (총 87개) :
  - thing: 7
  - experience: 6
  - comfortable: 5
  - word: 4
  - staff: 3
  - choice: 3
  - price: 3
  - comment: 3
  - room: 3
  - breakfast: 2
--------------------------------------------------
'nice'이(가) 꾸며주는 명사 (총 8,854개) :
  - room: 1,168
  - staff: 979
  - location: 694
  - hotel: 658
  - breakfast: 499
  - touch: 378
  - view: 310
  - area: 248
  - bed: 237
  - bathroom: 192
--------------------------------------------------
'comfortable'이(가) 꾸며주는 명사 

```
갯수가 좀 적더라도 명사에 집중!

attraction_keywords = ['museum', 'park', 'view', 'beautiful', 'amazing']
eat_keywords = ['restaurant', 'bar']
transport_keywords = ['great', 'train', 'subway', 'metro', 'bus', 'station', 'metre', 'underground', 'transport', 'access', 'location', 'centre', 'position', 'tube']
service_keywords = ['lovely', 'friendly', 'staff', 'helpful', 'breakfast', 'tidy', 'polite', 'reception','value','quality','check','experience','response','food','drink','egg','selection','desk','day','coffee']
facility_amenity_kewords = ['water','hot','cold','condition','balcony', 'wifi','desk','hotel','room','bed','water','furniture','building','carpet','window','mattress','wifi','connection','elevator','lift','fridge','lighting','tub','shower','air','level','audio','recording','bathroom']
roomsize_keywords = ['tidy']

=================================================

- 'very' 형용사를 꾸며주는 역할이 너무 강해서 제외
- 'good' -> location/breakfast/staff ----------------------> transport,service
- 'bad' -> thing / experience -----------------------------> service
- 'nice'-> room/staff/location ----------------------------> facility_amenity, service, transport
- 'comfortable' -> bed/room/staff -------------------------> facility_amenity, service
- 'clean' -> room/staff/location --------------------------> service, transport
-'excellent' -> location/staff/breakfast ------------------> transport,service
- 'perfect' -> location/staff/hotel -----------------------> transport,service,facility_amenity
- 'quiet' -> room/location/staff --------------------------> transport,service
- 'really' 형용사를 꾸며주는 역할이 너무 강해서 제외
- 'modern' -> room/hotel/staff ----------------------------> facility_amenity, service
- 'spacious' -> room/staff/bathroom -----------------------> facility_amenity, service
- 'fantastic' -> location/staff/room ----------------------> transport, service, facility_amenity
- 'comfy' -> bed/room/staff -------------------------------> facility_amenity, service
- 'so' 형용사를 꾸며주는 역할이 너무 강해서 제외

====================================================

결과적으로 상위 3개 안에 드는 명사 중 카테고리 분류가 나지 않은 것

1. thing - 추상명사라 카테고리에 추가 X
2. bathroom - facility_amenity에 추가
나머지는 이미 분류가 되었으므로 넣지 않음


#. other 처리 추가!

- positive_reviews_bert_analysis_1 파일 이용
- other 카테고리에서 상위권에 있는, 또는 의미가 명확한 단어들을 뽑아서 기존 카테고리에 추가

In [ ]:
df_pairs = pd.read_csv("/content/noun_adj_pairs_raw(pos).csv")

processed_keyword_categories = {
    'attraction': attraction_keywords,
    'eat': eat_keywords,
    'transport': transport_keywords,
    'service': service_keywords,
    'roomsize': roomsize_keywords,
    'facility_amenity' : facility_amenity_kewords
}


# 카테고리 라벨링 함수 정의
def assign_category_by_noun_priority(adj, noun):
    categories = []
    adj_lower = str(adj).lower() if pd.notna(adj) else ''
    noun_lower = str(noun).lower() if pd.notna(noun) else ''

    # 1순위: 명사(noun)가 어떤 키워드에 속하는지 확인
    for category, keywords in processed_keyword_categories.items():
        if any(kw in noun_lower for kw in keywords):
            if category not in categories:
                categories.append(category)

    # 2순위: 형용사(adj)가 명확한 카테고리 키워드일 때 (and 아직 추가되지 않은 카테고리에 한해)
    for category, keywords in processed_keyword_categories.items():
        if any(kw in adj_lower for kw in keywords):
            if category not in categories:
                categories.append(category)

    # 만약 아무 카테고리도 할당되지 않았다면 'other'
    return categories if categories else ['other']


# 5. 데이터프레임에 카테고리 컬럼 추가
df_pairs['categories'] = df_pairs.apply(lambda row: assign_category_by_noun_priority(row['adj'], row['noun']), axis=1)

# 6. 빈도수 계산
pair_counts = Counter(tuple(row) for row in df_pairs[['adj', 'noun']].values)

# 7. 'other' 카테고리 상위 쌍 확인
other_pairs_df = df_pairs[df_pairs['categories'].apply(lambda x: 'other' in x)].copy()

if not other_pairs_df.empty:
    other_pairs_df['count'] = other_pairs_df.apply(lambda row: pair_counts.get((row['adj'], row['noun']), 0), axis=1)
    print("\n### (참고) 라벨링 되지 않은 'other' 카테고리 상위 쌍 (빈도수 기준) ###")
    print(other_pairs_df.sort_values(by='count', ascending=False).head(10).to_string())
    print("----------------------------------------------------------------")

    distinct_other_pairs = other_pairs_df[['adj', 'noun']].drop_duplicates()
    print("\n### 라벨링 되지 않은 'other' 카테고리의 고유한 쌍들 (Distinct Pairs) ###")
    if len(distinct_other_pairs) > 10:
        print(f"총 {len(distinct_other_pairs)}개의 고유한 쌍 중 상위 10개:")
        print(distinct_other_pairs.head(10).to_string(index=False))
    else:
        print(f"총 {len(distinct_other_pairs)}개의 고유한 쌍:")
        print(distinct_other_pairs.to_string(index=False))

else:
    print("\n❗ 'other' 카테고리에 속하는 단어 쌍이 없습니다.")

print("\n---")
print("🎉 최종 라벨링 결과 (카테고리 컬럼 추가된 버전) 🎉")
print(df_pairs.head(10).to_string())

print("\n📈 각 카테고리별 쌍의 개수 📈")
all_assigned_categories = [cat for sublist in df_pairs['categories'] for cat in sublist]
print(Counter(all_assigned_categories))


### (참고) 라벨링 되지 않은 'other' 카테고리 상위 쌍 (빈도수 기준) ###
              adj     noun categories  count
391372  excellent  service    [other]   2153
643237  excellent  service    [other]   2153
44030   excellent  service    [other]   2153
390964  excellent  service    [other]   2153
643339  excellent  service    [other]   2153
522393  excellent  service    [other]   2153
44261   excellent  service    [other]   2153
277471  excellent  service    [other]   2153
522892  excellent  service    [other]   2153
523114  excellent  service    [other]   2153
----------------------------------------------------------------

### 라벨링 되지 않은 'other' 카테고리의 고유한 쌍들 (Distinct Pairs) ###
총 50157개의 고유한 쌍 중 상위 10개:
     adj      noun
    many    people
    true    beauty
  little     place
  unique structure
    lush  greenery
 awesome    arrive
  gentle       man
 spanish       man
    easy       way
relevant    ticket

---
🎉 최종 라벨링 결과 (카테고리 컬럼 추가된 버전) 🎉
        adj       noun    categories
0      good   location  


```
(중복가능! + 명사에 집중하기!)

attraction_keywords = ['museum', 'park', 'view', 'beautiful', 'amazing']
eat_keywords = ['restaurant', 'bar']
transport_keywords = ['great', 'train', 'subway', 'metro', 'bus', 'station', 'metre', 'underground', 'transport', 'access', 'location', 'centre', 'position', 'tube']
service_keywords = ['lovely', 'friendly', 'staff', 'helpful', 'breakfast', 'tidy', 'polite', 'reception','value','quality','check','experience','response','food','drink','egg','selection','desk','day','coffee']
facility_amenity_kewords = ['water','hot','cold','condition','balcony', 'wifi','desk','hotel','room','bed','water','furniture','building','carpet','window','mattress','wifi','connection','elevator','lift','fridge','lighting','tub','shower','air','level','audio','recording','bathroom']
roomsize_keywords = ['tidy']



service ----------------> service 카테고리에 추가
beauty -----------------> attraction 카테고리에 추가 (풍경)
structure --------------> attraction, facility_amenity 카테고리에 추가
greenery ---------------> attraction (풍경) 카테고리에 추가
ticket -----------------> transport, attraction 카테고리에 추가

===================================================



[최종 가버전1]

attraction_keywords = ['museum', 'park', 'view', 'beautiful', 'amazing', 'beauty', 'structure', 'greenery','ticket']
eat_keywords = ['restaurant', 'bar']
transport_keywords = ['great', 'train', 'subway', 'metro', 'bus', 'station', 'metre', 'underground', 'transport', 'access', 'location', 'centre', 'position', 'tube','ticket']
service_keywords = ['lovely', 'friendly', 'staff', 'helpful', 'breakfast', 'tidy', 'polite', 'reception','value','quality','check','experience','response','food','drink','egg','selection','desk','day','coffee','service']
facility_amenity_kewords = ['water','hot','cold','condition','balcony', 'wifi','desk','hotel','room','bed','water','furniture','building','carpet','window','mattress','wifi','connection','elevator','lift','fridge','lighting','tub','shower','air','level','audio','recording','bathroom','structure']
roomsize_keywords = ['tidy']




